In [1]:
import pandas as pd

In [2]:
pd.options.display.max_columns = 88

In [15]:
# Cambiar ruta a la oficial Google Cloud para no depender del local

df = pd.read_csv("https://storage.googleapis.com/datamexico-data/inegi_intercensal_census/TR_PERSONA01.CSV", 
                 encoding="latin-1",
                 dtype=str,
                 low_memory=False)

In [16]:
# Coordinar columnas con sheets
df.columns = map(str.lower, df.columns)

In [17]:
df.head()

,id_viv,id_persona,ent,nom_ent,mun,nom_mun,loc50k,nom_loc,cobertura,estrato,upm,factor,numper,sexo,edad,parent,parent_otro_c,ident_madre,ident_padre,sersalud,afrodes,acta_nac,dhsersal1,dhsersal2,perte_indigena,ent_pais_nac,nacionalidad,hlengua,qdialect_c,qdialect_inali,hespanol,elengua,asisten,mun_asi,nom_mun_asi,ent_pais_asi,tie_traslado_escu,med_traslado_esc1,med_traslado_esc2,med_traslado_esc3,escolari,nivacad,alfabet,escoacum,mun_res10,nom_mun_res10,ent_pais_res10,situa_conyugal,ident_pareja,conact,ocupacion_c,situacion_trab,aguinaldo,vacaciones,servicio_medico,utilidades,incap_sueldo,sar_afore,credito_vivienda,ingtrmen,actividades_c,mun_trab,nom_mun_trab,ent_pais_trab,tie_traslado_trab,med_traslado_trab1,med_traslado_trab2,med_traslado_trab3,acti_sin_pago1,acti_sin_pago2,acti_sin_pago3,acti_sin_pago4,acti_sin_pago5,acti_sin_pago6,acti_sin_pago7,acti_sin_pago8,hijos_nac_vivos,hijos_fallecidos,hijos_sobreviv,fecha_nac_m,fecha_nac_a,sobrevivencia,edad_morir_d,edad_morir_m,edad_morir_a,tamloc
0,10010000001,1001000000102,01,Aguascalientes,001,Aguascalientes,0001,Aguascalientes,2,01-001-05,000041,15,2,3,35,2,201,96,97,5,3,1,1,NaN,1,001,5,3,NaN,NaN,NaN,3,7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6,2,5,6,001,Aguascalientes,001,1,1,33,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,40,40,0,12,15,1,4,0,4,4,2009,1,NaN,NaN,NaN,5
1,10010000001,1001000000103,01,Aguascalientes,001,Aguascalientes,0001,Aguascalientes,2,01-001-05,000041,15,3,3,12,3,301,2,1,5,3,1,1,NaN,1,001,5,3,NaN,NaN,NaN,3,5,001,Aguascalientes,001,1,6,NaN,NaN,4,2,5,4,001,Aguascalientes,001,6,NaN,31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0,0,0,5,1,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5
2,10010000001,1001000000104,01,Aguascalientes,001,Aguascalientes,0001,Aguascalientes,2,01-001-05,000041,15,4,1,5,3,301,2,1,5,3,1,1,NaN,1,001,5,3,NaN,NaN,NaN,3,5,001,Aguascalientes,001,2,6,NaN,NaN,2,1,7,0,001,Aguascalientes,001,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5
3,10010000001,1001000000101,01,Aguascalientes,001,Aguascalientes,0001,Aguascalientes,2,01-001-05,000041,15,1,1,37,1,101,96,96,5,3,1,1,NaN,1,032,5,3,NaN,NaN,NaN,3,7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6,2,5,6,001,Aguascalientes,001,1,2,10,834,1,1,3,5,7,1,3,5,3857,4611,001,Aguascalientes,001,1,6,NaN,NaN,0,0,15,20,0,0,7,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5
4,10010000002,1001000000206,01,Aguascalientes,001,Aguascalientes,0001,Aguascalientes,2,01-001-06,000136,35,6,1,5,3,301,2,1,1,3,1,2,NaN,3,001,5,3,NaN,NaN,NaN,3,5,001,Aguascalientes,001,1,6,NaN,NaN,0,0,9,0,001,Aguascalientes,001,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5


In [18]:
# Agregar columnas de ids y rename de factor a population
df["mun_id"] = df["ent"] + df["mun"]
df["loc_id"] = df["mun_id"] + df["loc50k"]
df.rename(index=str, columns={"factor":"population"}, inplace=True)

In [19]:
# Convertir elementos necesarios a int
df["mun_id"] = df["mun_id"].astype(int)
df["loc_id"] = df["loc_id"].astype(int)
df["population"] = df["population"].astype(int)

In [20]:
# Separar columnas en 3 listas: 
                                # values, columnas de enteros
                                # params, parametros a trabajar
                                # sides, columnas en el spreedsheet no agregadas todavia

values = [
    "mun_id",
    "loc_id",
    "population"
]

params = [
    "sexo",
    "parent",
    "sersalud",
    "dhsersal1",
    "nacionalidad"
]

sides = [
    "ident_madre",
    "ident_padre",
    "tamloc",
    "conact"
]

In [21]:
# Cargar spreedsheet para realizar los cambios a la tabla de Personas
df_labels = pd.ExcelFile("https://docs.google.com/spreadsheets/d/e/2PACX-1vR08Js9Sh4nNTMe5uBcsDUFedG5MOjIf90p6EHAr1_sWY5kpnI3xUvyPHzQpTEUrXz1pskaoc0uyea6/pub?output=xlsx")

for sheet in params:
    df_l = pd.read_excel(df_labels, sheet)
    print(sheet)
    df[sheet] = df[sheet].astype(int)
    df[sheet] = df[sheet].replace(dict(zip(df_l.prev_id, df_l.id)))
 

sexo
parent
sersalud
dhsersal1
nacionalidad


In [22]:
df[params].head()

,sexo,parent,sersalud,dhsersal1,nacionalidad
0,2,2,5,1,1
1,2,3,5,1,1
2,1,3,5,1,1
3,1,1,5,1,1
4,1,3,1,2,1


In [23]:
df = df.groupby(params + ["mun_id", "loc_id"]).sum().reset_index()

In [24]:
df

,sexo,parent,sersalud,dhsersal1,nacionalidad,mun_id,loc_id,population
0,1,1,0,0,0,1001,10010001,24
1,1,1,0,0,0,1007,10070000,4
2,1,1,0,0,1,1009,10090000,2
3,1,1,0,1,1,1002,10020000,7
4,1,1,0,1,1,1003,10030000,4
5,1,1,0,1,1,1007,10070000,4
6,1,1,0,1,1,1008,10080000,4
7,1,1,0,1,1,1010,10100000,2
8,1,1,0,2,1,1001,10010000,9
9,1,1,0,2,1,1005,10050000,16
